# Cell types and clustering

(working backwards from a mind map that I made)



At the end of the course, students should be able to...

#### List some definitions of cell types

morphological, electrophysiological, evolutionary, transcriptional, etc. Encourage them to find more!

#### Explain the principle of clustering and why it is so important for unsupervised analysis

improve signal/noise ratio?

#### List ways of calculating cell-cell similarity, explain why this might be important

different metrics
- cosine
- euclidean
- correlation
- others?

different feature subsets
- all genes/HVG
- raw/normalised
- transformed (log1p/sqrt, etc.)

different embeddings
- PCA
- CCA
- LLE
- diffusion maps
- NMF
- raw
- spectral decomposition
- UMAP
- tSNE

introduce them to the idea of negative tests! Why might UMAP/tSNE be misleading? Show circle examples, use simple datasets from visualisation paper.

#### explain their preference for one

math makes sense/looks good/fast/agrees with what we know/everyone does it/???

#### Produce a cell type family tree

Calculate and plot a bootstrapped neighbour-joining tree of all cell type clusters.

#### Use a cell type tree to identify cell type families

Explain which parts of the tree they trust and which not. Explain why they kept or excluded certain clusters. Use tree to define cell type families. Produce dotplot of CT family characteristic gene expression, combine with literature evidence.